## Import Libraries

In [46]:
import os
import time
import random

import math
from math import ceil

import numpy as np
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt

from spectrum import clear_spectrum
from traffic import generate_traffic
from algorithms.utils import k_shortest_paths

from algorithms.genetic_algorithm import GenAlg
from algorithms.first_fit import k_SP_FF_RSA

### Fix seed

In [47]:
seed = 9
random.seed(seed)
np.random.seed(seed)

## Simulation Parameters

In [48]:
# Select the topology
TOPOLOGY = 'National' # 'Continental'
# Select spectrum partitioning
PARTITIONING = 'Hard' # 'Hard'
# Select the algorithm to be used
ALGORITHM = 'FF' # 'FF'

# Density of the network (wrt fully connected graph)
density = 0.5
# Capacity of the network links (THz)
CAPACITY = 4.4*(10**3) 
# Slice width (GHz) (used for section of 50 GHz (2 slices) or 75 GHz (3 slices))
slot_width = 25 
# Number of slots in the network
num_slots = int(CAPACITY/slot_width)

# Initial, maximum and relative number of demands
num_demands_init = 350 
num_demands_max = 2000
num_demands_delta = 50
# Number of shortest paths to be considered
k = 4 
# Number of Monte Carlo simulations
MC = 100

### Set secondary parameters according to the experiment

In [ ]:
####### TOPOLOGY PARAMETERS #######
if TOPOLOGY == 'Continental':
    print('Continental topology')
    # Continental topology
    Nodes = 7 # Number of nodes
    d_min, d_max = 400, 1100 # Min and max distances between nodes
    d_max_req, d_avg_req = 1000, 300 # Max and average distances (REQUIRED)
    # A requirement for the number of links
    req_links = lambda x: x>=20 
    
elif TOPOLOGY == 'National':
    print('National topology')
    # National topology
    Nodes = 4 # Number of nodes
    d_min, d_max = 150, 450 # Min and max distances between nodes
    d_max_req, d_avg_req = 400, 150 # Max and average distances (REQUIRED)    
    # A requirement for the number of links
    req_links = lambda x: x<=20 

if PARTITIONING == 'Soft':
    print('Soft partitioning')
    border = None
    last_slot_75 = None
elif PARTITIONING == 'Hard':
    print('Hard partitioning')
    # Percent of the spectrum belongs to 75 GHz, the rest is to 50 GHz
    border = 0.1 
    # Round to closest 3 slots
    last_slot_75 = 3*ceil(num_slots*border/3)

if ALGORITHM == 'GA':
    print('Genetic Algorithm')
    import logging
    # Disable logging output for the 'jmetal' logger
    logging.getLogger('jmetal').setLevel(logging.CRITICAL)
    algorithm = GenAlg
elif ALGORITHM == 'FF':
    print('First Fit Algorithm')
    algorithm = k_SP_FF_RSA

## Generate the topology

In [ ]:
# Generate the distances between nodes as weights of the edges
distances = np.random.randint(d_min, d_max, size=(Nodes,Nodes))

# Generate the adjacency matrix
mask = np.random.choice([0, 1], size=(Nodes,Nodes), p=[1-density, density])
a = np.multiply(distances, mask)

# Force the matrix to be symmetric
A = ((a + a.T)/2).astype(int)
# Remove self loops
np.fill_diagonal(A,0)

# Create the graph
G = nx.DiGraph(A)
    
N, E = list(G.nodes()), list(G.edges())
weights = nx.get_edge_attributes(G,'weight')# set of lengths of each edge
weight_avg = np.mean(list(weights.values()))
weight_max = np.max(list(weights.values()))

# Check if the topology satisfies the requirements
print('Nodes:', N)
print('Edges:', E)
print('Is the number of links enough?', req_links(len(E)), f'(Number of links: {len(E)})')
print(f'Avg. distance: {weight_avg:.2f}km, Max. distance: {weight_max}km')
print(f'Avg. distance (REQUIRED): ~{d_avg_req} km, Max. distance (REQUIRED): ~{d_max_req}km')

### Visualize the graph and the adjacency matrix

In [ ]:
print('Adjacency matrix:\n', A)
pos=nx.spring_layout(G, seed=seed)
nx.draw(G, pos, with_labels=True, font_weight='bold')
edge_weight = nx.get_edge_attributes(G,'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels = edge_weight)
plt.show()

### Useful Function

In [52]:
def confidence(array):
    """Compute the confidence interval"""
    return np.std(array) * 1.96 / math.sqrt(len(array)) 

## Algorithms simulation

Simulation is performed over a constantly increasing number of demands. For each of the number of demands, the results of multiple Monte-Carlo simulations are averaged and stored.

In [ ]:
# Create a dataframe to store the results
columns = ['traff_mean', 'num_demands', 'bp_mean', 'bp_conf', 'cost_mean', 'cost_conf']
df = pd.DataFrame(columns=columns)

start_time0 = time.time()
# Loop over the number of demands
for num_demands in range(num_demands_init, num_demands_max, num_demands_delta):
    print(num_demands)
    traffic, bp, cost = [], [], []

    start_time = time.time()
    # Loop over the Monte Carlo simulations
    for m in range(MC):
        # Generate the traffic demands
        D = generate_traffic(num_demands, Nodes)
        # Compute the total offered traffic (in Gbps)
        offered_traff = sum(sum(x) for x in D.values()) 
        traffic.append(offered_traff)
        clear_spectrum(G, num_slots)

        # Compute the k shortest paths
        paths_dict, len_p = k_shortest_paths(G, N, k)
        # Compute the blocking probability and the number of regenerators 
        # by solving the RSA problem with selected algorithm
        num_reg, lost_traffic = algorithm(G, k, paths_dict, len_p, D, last_slot_75, PARTITIONING)
        
        # Compute the blocking probability
        bp.append(lost_traffic/offered_traff)
        cost.append(num_reg)

    # Store the results 
    bp_current = np.mean(bp)   
    row = [offered_traff, num_demands, np.mean(bp), confidence(bp), np.mean(cost), confidence(cost)]    
    df.loc[len(df)] = row
    end_time = time.time()
    
    print("Time to compute a demand is {:.2f} min".format((end_time-start_time)/60))
    print(bp_current)
    
    # Stop the simulation if the blocking probability is higher than 1%
    if bp_current > 0.01:
        break
end_time0 = time.time()
print("Total time to compute is {:.2f} min".format((end_time0-start_time0)/60))


## Save the results in a CSV file

In [54]:
# Save the results
name = ALGORITHM + '_' + PARTITIONING + '_' + TOPOLOGY

# Append the border value hyperparameter if hard partitioning is used
if PARTITIONING == 'Hard':
    name+= ('_' + str(border))

os.makedirs('results', exist_ok=True)
df.to_csv('results/'+name+'.csv', index=False)